In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import data

In [59]:
verbose = False
path = './data/2013.txt'
corpus = data.Corpus(path, verbose)

In [122]:
class Model(nn.Module):
        
    def __init__(self, vocab_size, embed_size, nhidden, nlayers):
        super(Model, self).__init__()
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, nhidden, nlayers)
        self.decoder = nn.Linear(nhidden, vocab_size)

    def forward(self, x, h0):
        y = self.encoder(x)
        y = self.rnn(x, h0)
        y = self.decoder(y)
        #output = F.log_softmax(output, dim=0)
        return output

In [125]:
vocab_size = len(corpus)
embed_size = 200
nhidden = 10
nlayers = 2
lm = Model(vocab_size, embed_size, nhidden, nlayers)
lm
# for i in range(10):
#     out = lm(corpus.data[i].to(torch.long), torch.tensor(i)).argmax()
#     print(corpus.ix2word[out])

Model(
  (encoder): Embedding(27738, 200)
  (rnn): RNN(200, 10, num_layers=2)
  (decoder): Linear(in_features=10, out_features=27738, bias=True)
)